In [32]:
import os
from dataclasses import dataclass
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from config import (
    countries,
    dap_bidding_zones,
    interconnections,
    interconnections_edge_matrix,
)
from tqdm import tqdm
from dotenv import load_dotenv

import math
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv
from torch_geometric.data import Data
from torch.nn import BatchNorm1d

In [33]:
load_dotenv()
engine = create_engine(os.getenv("SQLALCHEMY_DATABASE_URI"))

In [34]:
flow_df = pd.read_sql_table("flow_32", engine)
flow_df = flow_df.set_index("DateTime")
flow_df.fillna(0, inplace=True)

In [35]:
dap_df = pd.DataFrame()
for country_id in countries.keys():
    dap_df[country_id] = pd.read_sql_table(f"{country_id}_dap", engine).set_index(
        "DateTime"
    )
dap_df.index = pd.to_datetime(dap_df.index)
dap_df.ffill(inplace=True)
dap_df.fillna(0, inplace=True)

In [36]:
load_df = pd.DataFrame()
for country_id in countries.keys():
    load_df[country_id] = pd.read_sql_table(f"{country_id}_load", engine).set_index(
        "DateTime"
    )
load_df.ffill(inplace=True)
# Fille NaN with mean of the column
load_df.fillna(load_df.mean(), inplace=True)

In [37]:
biomass_df = pd.DataFrame()
fossil_brown_coal_df = pd.DataFrame()
fossil_coal_derived_gas_df = pd.DataFrame()
fossil_gas_df = pd.DataFrame()
fossil_hard_coal_df = pd.DataFrame()
fossil_oil_df = pd.DataFrame()
hydro_pumped_storage_df = pd.DataFrame()
hydro_run_of_river_and_poundage_df = pd.DataFrame()
hydro_water_reservoir_df = pd.DataFrame()
nuclear_df = pd.DataFrame()
other_df = pd.DataFrame()
other_renewable_df = pd.DataFrame()
solar_df = pd.DataFrame()
waste_df = pd.DataFrame()
wind_offshore_df = pd.DataFrame()
wind_onshore_df = pd.DataFrame()
geothermal_df = pd.DataFrame()
fossil_peat_df = pd.DataFrame()

gen_types = [
    "Biomass",
    "Fossil Brown coal/Lignite",
    "Fossil Coal-derived gas",
    "Fossil Gas",
    "Fossil Hard coal",
    "Fossil Oil",
    "Hydro Pumped Storage",
    "Hydro Run-of-river and poundage",
    "Hydro Water Reservoir",
    "Nuclear",
    "Other",
    "Other renewable",
    "Solar",
    "Waste",
    "Wind Offshore",
    "Wind Onshore",
    "Geothermal",
    "Fossil Peat",
]

for country_id in countries.keys():
    this_cty_gen_df = pd.read_sql_table(f"{country_id}_gen", engine).set_index(
        "DateTime"
    )
    biomass_df[country_id] = this_cty_gen_df["Biomass"]
    fossil_brown_coal_df[country_id] = this_cty_gen_df["Fossil Brown coal/Lignite"]
    fossil_coal_derived_gas_df[country_id] = this_cty_gen_df["Fossil Coal-derived gas"]
    fossil_gas_df[country_id] = this_cty_gen_df["Fossil Gas"]
    fossil_hard_coal_df[country_id] = this_cty_gen_df["Fossil Hard coal"]
    fossil_oil_df[country_id] = this_cty_gen_df["Fossil Oil"]
    hydro_pumped_storage_df[country_id] = this_cty_gen_df["Hydro Pumped Storage"]
    hydro_run_of_river_and_poundage_df[country_id] = this_cty_gen_df[
        "Hydro Run-of-river and poundage"
    ]
    hydro_water_reservoir_df[country_id] = this_cty_gen_df["Hydro Water Reservoir"]
    nuclear_df[country_id] = this_cty_gen_df["Nuclear"]
    other_df[country_id] = this_cty_gen_df["Other"]
    other_renewable_df[country_id] = this_cty_gen_df["Other renewable"]
    solar_df[country_id] = this_cty_gen_df["Solar"]
    waste_df[country_id] = this_cty_gen_df["Waste"]
    wind_offshore_df[country_id] = this_cty_gen_df["Wind Offshore"]
    wind_onshore_df[country_id] = this_cty_gen_df["Wind Onshore"]
    geothermal_df[country_id] = this_cty_gen_df["Geothermal"]
    fossil_peat_df[country_id] = this_cty_gen_df["Fossil Peat"]

biomass_df.fillna(0, inplace=True)
fossil_brown_coal_df.fillna(0, inplace=True)
fossil_coal_derived_gas_df.fillna(0, inplace=True)
fossil_gas_df.fillna(0, inplace=True)
fossil_hard_coal_df.fillna(0, inplace=True)
fossil_oil_df.fillna(0, inplace=True)
hydro_pumped_storage_df.fillna(0, inplace=True)
hydro_run_of_river_and_poundage_df.fillna(0, inplace=True)
hydro_water_reservoir_df.fillna(0, inplace=True)
nuclear_df.fillna(0, inplace=True)
other_df.fillna(0, inplace=True)
other_renewable_df.fillna(0, inplace=True)
solar_df.fillna(0, inplace=True)
waste_df.fillna(0, inplace=True)
wind_offshore_df.fillna(0, inplace=True)
wind_onshore_df.fillna(0, inplace=True)
geothermal_df.fillna(0, inplace=True)
fossil_peat_df.fillna(0, inplace=True)

In [38]:
datetime_intersect = (
    flow_df.index.intersection(dap_df.index)
    .intersection(load_df.index)
    .intersection(biomass_df.index)
    .intersection(fossil_brown_coal_df.index)
    .intersection(fossil_coal_derived_gas_df.index)
    .intersection(fossil_gas_df.index)
    .intersection(fossil_hard_coal_df.index)
    .intersection(fossil_oil_df.index)
    .intersection(hydro_pumped_storage_df.index)
    .intersection(hydro_run_of_river_and_poundage_df.index)
    .intersection(hydro_water_reservoir_df.index)
    .intersection(nuclear_df.index)
    .intersection(other_df.index)
    .intersection(other_renewable_df.index)
    .intersection(solar_df.index)
    .intersection(waste_df.index)
    .intersection(wind_offshore_df.index)
    .intersection(wind_onshore_df.index)
    .intersection(geothermal_df.index)
    .intersection(fossil_peat_df.index)
)
print(len(datetime_intersect))
print(min(datetime_intersect), max(datetime_intersect))
# Check if datetime_intersect is monotonically increasing
assert all(
    datetime_intersect[i] < datetime_intersect[i + 1]
    for i in range(len(datetime_intersect) - 1)
)

43729
2015-01-04 23:00:00 2019-12-31 23:00:00


In [39]:
# Create temporal features based on datetime_intersect
temporal_hour_df = pd.DataFrame(index=datetime_intersect)
temporal_dow_df = pd.DataFrame(index=datetime_intersect)
temporal_month_df = pd.DataFrame(index=datetime_intersect)
temporal_doy_df = pd.DataFrame(index=datetime_intersect)
for country_id in countries.keys():
    temporal_hour_df[country_id] = datetime_intersect.hour
    temporal_dow_df[country_id] = datetime_intersect.dayofweek
    temporal_month_df[country_id] = datetime_intersect.month
    temporal_doy_df[country_id] = datetime_intersect.dayofyear

In [40]:
flow_df = flow_df.loc[datetime_intersect]
dap_df = dap_df.loc[datetime_intersect]
load_df = load_df.loc[datetime_intersect]
biomass_df = biomass_df.loc[datetime_intersect]
fossil_brown_coal_df = fossil_brown_coal_df.loc[datetime_intersect]
fossil_coal_derived_gas_df = fossil_coal_derived_gas_df.loc[datetime_intersect]
fossil_gas_df = fossil_gas_df.loc[datetime_intersect]
fossil_hard_coal_df = fossil_hard_coal_df.loc[datetime_intersect]
fossil_oil_df = fossil_oil_df.loc[datetime_intersect]
hydro_pumped_storage_df = hydro_pumped_storage_df.loc[datetime_intersect]
hydro_run_of_river_and_poundage_df = hydro_run_of_river_and_poundage_df.loc[
    datetime_intersect
]
hydro_water_reservoir_df = hydro_water_reservoir_df.loc[datetime_intersect]
nuclear_df = nuclear_df.loc[datetime_intersect]
other_df = other_df.loc[datetime_intersect]
other_renewable_df = other_renewable_df.loc[datetime_intersect]
solar_df = solar_df.loc[datetime_intersect]
waste_df = waste_df.loc[datetime_intersect]
wind_offshore_df = wind_offshore_df.loc[datetime_intersect]
wind_onshore_df = wind_onshore_df.loc[datetime_intersect]
geothermal_df = geothermal_df.loc[datetime_intersect]
fossil_peat_df = fossil_peat_df.loc[datetime_intersect]

In [41]:
edges = np.array(interconnections_edge_matrix)
print(edges.shape)
# Map edge names to indices
edge_names = np.unique(edges)
edge_map = {edge: i for i, edge in enumerate(edge_names)}
edge_indices = np.array([edge_map[edge] for edge in edges.flatten()]).reshape(
    edges.shape
)
# Repeat edge indices for each datetime
edge_indices = np.repeat(
    edge_indices[np.newaxis, :, :],
    len(datetime_intersect),
    axis=0,
)
print(edge_indices.shape)
n_edges = edges.shape[1]

(2, 32)
(43729, 2, 32)


In [42]:
# Edge labels (flow) of shape (n_datetime, n_edges, 1)
edge_labels = np.array(flow_df)
# print(edge_labels.shape)
edge_labels = np.reshape(
    edge_labels, (len(datetime_intersect), edge_labels.shape[1], 1)
)
print(edge_labels.shape)
edge_attributes = np.copy(edge_labels)
print(edge_attributes.shape)

(43729, 32, 1)
(43729, 32, 1)


In [43]:
# Node features (dap, load) of shape (n_datetime, n_nodes, n_node_features)
node_features = np.stack(
    [
        dap_df.to_numpy(),
        load_df.to_numpy(),
        biomass_df.to_numpy(),
        fossil_brown_coal_df.to_numpy(),
        fossil_coal_derived_gas_df.to_numpy(),
        fossil_gas_df.to_numpy(),
        fossil_hard_coal_df.to_numpy(),
        fossil_oil_df.to_numpy(),
        hydro_pumped_storage_df.to_numpy(),
        hydro_run_of_river_and_poundage_df.to_numpy(),
        hydro_water_reservoir_df.to_numpy(),
        nuclear_df.to_numpy(),
        other_df.to_numpy(),
        other_renewable_df.to_numpy(),
        solar_df.to_numpy(),
        waste_df.to_numpy(),
        wind_offshore_df.to_numpy(),
        wind_onshore_df.to_numpy(),
        geothermal_df.to_numpy(),
        fossil_peat_df.to_numpy(),
        temporal_hour_df.to_numpy(),
        temporal_dow_df.to_numpy(),
        temporal_month_df.to_numpy(),
        temporal_doy_df.to_numpy(),
    ],
    axis=-1,
)
print(node_features.shape)
print(node_features[0, 0, :])
# print(node_features)
n_nodes = node_features.shape[1]

(43729, 10, 24)
[3.656000e+01 1.003953e+04 2.311300e+02 0.000000e+00 0.000000e+00
 1.791710e+03 4.368500e+02 0.000000e+00 0.000000e+00 3.596000e+01
 0.000000e+00 3.904350e+03 6.016400e+02 0.000000e+00 0.000000e+00
 2.690100e+02 4.937000e+01 2.415900e+02 0.000000e+00 0.000000e+00
 2.300000e+01 6.000000e+00 1.000000e+00 4.000000e+00]


In [44]:
assert (
    len(datetime_intersect)
    == edge_indices.shape[0]
    == edge_labels.shape[0]
    == edge_attributes.shape[0]
    == node_features.shape[0]
)

In [45]:
# Print a snapshot of the graph data
idx = 0
print(datetime_intersect[idx])
print(edge_indices[idx])
print(edge_labels[idx])
print(edge_attributes[idx])
print(node_features[idx])

2015-01-04 23:00:00
[[0 0 0 0 3 3 3 4 4 4 4 2 2 2 2 2 5 6 6 7 7 7 7 7 8 8 1 1 9 9 9 9]
 [4 6 7 9 2 7 8 0 2 1 9 3 4 6 7 1 9 0 2 0 3 2 8 9 3 7 4 2 0 4 5 7]]
[[   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [1315.79]
 [  52.  ]
 [ 617.  ]
 [ 279.  ]
 [1433.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [3205.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [2106.86]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [ 964.  ]
 [   0.  ]
 [ 704.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [ 169.19]
 [   0.  ]]
[[   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [1315.79]
 [  52.  ]
 [ 617.  ]
 [ 279.  ]
 [1433.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [3205.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [2106.86]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [ 964.  ]
 [   0.  ]
 [ 704.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [   0.  ]
 [ 169.19]
 [   0.  ]]
[[3.65600000e+01 1.00395300e+04 2.31130000e+02 0.00000000e+00
  0.00000000e+00 1.79171000e+03 4.36850000e+02 0.00000000e+00
  0.00000000e+0

In [46]:
# https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GATv2Conv.html
class GNNEncoder(nn.Module):
    def __init__(
        self, hidden_channels, num_heads_GAT, dropout_p_GAT, edge_dim_GAT, momentum_GAT
    ):
        super().__init__()
        self.gat = GATv2Conv(
            (-1, -1),
            hidden_channels,
            add_self_loops=False,
            heads=num_heads_GAT,
            edge_dim=edge_dim_GAT,
        )
        self.norm = BatchNorm1d(
            hidden_channels,
            momentum=momentum_GAT,
            affine=False,
            track_running_stats=False,
        )
        self.dropout = nn.Dropout(dropout_p_GAT)

    def forward(self, x, edge_indices, edge_attrs):
        x = self.dropout(x)
        x = self.norm(x)
        nodes_embedds = self.gat(x, edge_indices, edge_attrs)
        nodes_embedds = F.leaky_relu(nodes_embedds, negative_slope=0.1)
        return nodes_embedds

In [47]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x: Tensor) -> Tensor:
        x = x + self.pe[: x.size(0)]
        return self.dropout(x)

In [48]:
class Transformer(nn.Module):
    def __init__(
        self,
        dim_model,
        num_heads_TR,
        num_encoder_layers_TR,
        num_decoder_layers_TR,
        dropout_p_TR,
    ):
        super().__init__()
        self.pos_encoder = PositionalEncoding(dim_model)
        self.transformer = nn.Transformer(
            d_model=dim_model,
            nhead=num_heads_TR,
            num_decoder_layers=num_encoder_layers_TR,
            num_encoder_layers=num_decoder_layers_TR,
            dropout=dropout_p_TR,
        )

    def forward(self, src, trg):
        src = self.pos_encoder(src)
        trg = self.pos_encoder(trg)
        temporal_node_embeddings = self.transformer(src, trg)
        return temporal_node_embeddings

In [49]:
class EdgeDecoder(nn.Module):
    def __init__(self, hidden_channels, num_heads_GAT, num_edges, num_nodes):
        super().__init__()
        self.lin1 = nn.Linear(
            num_nodes * hidden_channels * num_heads_GAT, hidden_channels
        )
        self.lin2 = nn.Linear(hidden_channels, num_edges)

    def forward(self, x):
        # Flatten the tensor
        x = torch.flatten(x)
        x = self.lin1(x)
        x = F.leaky_relu(x, negative_slope=0.1)
        x = self.lin2(x)
        return x.view(-1)

In [50]:
class Model(nn.Module):
    def __init__(
        self,
        hidden_channels,
        num_heads_GAT,
        dropout_p_GAT,
        edge_dim_GAT,
        momentum_GAT,
        dim_model,
        num_heads_TR,
        num_encoder_layers_TR,
        num_decoder_layers_TR,
        dropout_p_TR,
        n_edges,
        n_nodes,
    ):
        super().__init__()
        self.encoder = GNNEncoder(
            hidden_channels, num_heads_GAT, dropout_p_GAT, edge_dim_GAT, momentum_GAT
        )  # node embedding with GAT
        self.transformer = Transformer(
            dim_model,
            num_heads_TR,
            num_encoder_layers_TR,
            num_decoder_layers_TR,
            dropout_p_TR,
        )
        self.decoder = EdgeDecoder(hidden_channels, num_heads_GAT, n_edges, n_nodes)

    def forward(self, x, edge_indices, edge_attrs):
        src_embedds = []
        for i in range(x.shape[0]):
            src_embedds.append(self.encoder(x[i], edge_indices[i], edge_attrs[i]))
        src_embedds = torch.stack(src_embedds)
        trg_embedds = src_embedds[-1].unsqueeze(0)
        temporal_node_embedds = self.transformer(src_embedds, trg_embedds)
        temporal_node_embedds = temporal_node_embedds.squeeze(0)
        edge_predictions = self.decoder(temporal_node_embedds)
        return edge_predictions

In [51]:
def train(model, data, window_size, num_epochs, lr):
    model = model.to(device)
    data = [d.to(device) for d in data]
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    for epoch in range(num_epochs):
        loss_sum = None
        for m in tqdm(range(len(data) - window_size)):
            # for m in range(len(data) - window_size):
            optimizer.zero_grad()
            x = torch.stack([data[m + i].x for i in range(window_size)])
            edge_indices = torch.stack(
                [data[m + i].edge_index for i in range(window_size)]
            )
            edge_attrs = torch.stack(
                [data[m + i].edge_attr for i in range(window_size)]
            )
            y = data[m + window_size].y
            y_pred = model(x, edge_indices, edge_attrs)
            y_pred = y_pred.view((-1, 1))
            loss = criterion(y_pred, y)
            # print(f"Epoch {epoch}, Loss {loss.item()}")
            if loss_sum is None:
                loss_sum = loss
            else:
                loss_sum += loss
            # if m % 24 * 7 == 0 or m == len(data) - window_size - 1:
            if m == len(data) - window_size - 1 or m % 24 == 0:
                diff = y.squeeze() - y_pred
                diff = diff.detach().cpu().numpy()
                print(f"Epoch {epoch}, m={m}", diff.mean())
                loss_sum.backward()
                optimizer.step()
                optimizer.zero_grad()
                loss_sum = None

    return model

In [52]:
print(node_features.shape)
print(edge_indices.shape)
print(edge_attributes.shape)
print(edge_labels.shape)
snapshots = []
for i in range(len(datetime_intersect)):
    x = torch.tensor(node_features[i], dtype=torch.float)
    edge_index = torch.tensor(edge_indices[i], dtype=torch.long)
    edge_attr = torch.tensor(edge_attributes[i], dtype=torch.float)
    y = torch.tensor(edge_labels[i], dtype=torch.float)
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
    snapshots.append(data)
print(len(snapshots))

(43729, 10, 24)
(43729, 2, 32)
(43729, 32, 1)
(43729, 32, 1)
43729


In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model(
    hidden_channels=64,
    num_heads_GAT=4,
    dropout_p_GAT=0.1,
    edge_dim_GAT=1,  # edge attributes
    momentum_GAT=0.1,
    dim_model=64 * 4,  # hidden_channels * num_heads_GAT
    num_heads_TR=4,
    num_encoder_layers_TR=6,
    num_decoder_layers_TR=6,
    dropout_p_TR=0.1,
    n_edges=n_edges,
    n_nodes=n_nodes,
)
train(model, snapshots[:8760], window_size=24, num_epochs=10000, lr=0.001)

 79%|███████▉  | 6887/8736 [06:14<01:11, 26.01it/s]

Epoch 2, m=6888 100.416885


 79%|███████▉  | 6912/8736 [06:15<01:07, 27.13it/s]

Epoch 2, m=6912 100.10389


 79%|███████▉  | 6933/8736 [06:16<01:11, 25.32it/s]

...

Epoch 3, m=936 88.65631


 11%|█         | 959/8736 [00:51<05:01, 25.79it/s]

Epoch 3, m=960 101.14924


 11%|█▏        | 983/8736 [00:52<04:36, 27.99it/s]

Epoch 3, m=984 65.774345
